<center>
<h1>Time-Travel-Visual-Storytelling</h1>

<h4>Nelson Amaral, Nuno Cardoso, Tiago Santana </h4>
<h3>Tomar, Janeiro 2019</h3>
</center>

# Dependencies

* Python 3.7.x
* spacy
* json
* requests
* datetime
* contamehistorias
* relativedelta
* flask
* cors
* selenium
* OpenCV
* numpy

Instalar todas as dependencias necessarias (apenas a primeira vez)

In [ ]:
!pip install spacy
!python -m spacy download pt
!pip install json
!pip install requests
!pip install git+https://github.com/LIAAD/TemporalSummarizationFramework.git
!pip install python-dateutil
!pip install flask
!pip install flask-restful
!pip install cors
!pip install selenium
!pip install numpy
!pip install opencv-python

Em baixo importamos dependencias:

In [1]:
import spacy
import json
import requests
from contamehistorias.datasources.webarchive import ArquivoPT
from datetime import datetime
from contamehistorias import engine
import os  
from dateutil.relativedelta import relativedelta # $ pip install python-dateutil
from flask import Flask, render_template, send_from_directory
from flask_restful import Api, Resource, reqparse
from flask_cors import CORS
import cv2
import numpy as np
import urllib.request

Inicializamos a api:

In [2]:
app = Flask(__name__)
api = Api(app)
cors = CORS(app)

Adicionamos os dominios a ser consultados pelo contamehistorias

In [3]:
domains = ['http://acervo.publico.pt/'
,'http://inimigo.publico.pt/'
,'http://publico.pt/'
,'http://www.dn.pt/'
,'http://dn.sapo.pt/'
,'http://dnoticias.pt/'
,'http://www.rtp.pt/'
,'http://www.cmjornal.pt/'
,'http://www.iol.pt/'
,'http://www.tvi24.iol.pt/'
,'http://noticias.sapo.pt/'
,'http://www.sapo.pt/'
,'http://expresso.sapo.pt/'
,'http://sol.sapo.pt/'
,'http://visao.sapo.pt/'
,'http://exameinformatica.sapo.pt/'
,'http://tek.sapo.pt/'
,'http://www.jornaldenegocios.pt/'
,'http://dinheirodigital.sapo.pt/'
,'http://abola.pt/'
,'http://www.abola.pt/'
,'http://www.jn.pt/'
,'http://jn.pt/'
,'http://sicnoticias.sapo.pt/'
,'http://www.lux.iol.pt/'
,'http://maisfutebol.iol.pt/'
,'http://lux.iol.pt/'
,'http://www.ionline.pt/'
,'http://ionline.sapo.pt/'
,'http://news.google.pt/'
,'http://www.dinheirovivo.pt/'
,'http://www.aeiou.pt/'
,'http://zap.aeiou.pt/'
,'http://www.tsf.pt/'
,'http://meiosepublicidade.pt/'
,'http://www.sabado.pt/'
,'http://www.omirante.pt/'
,'http://www.jb.pt/'
,'http://www.mdb.pt/'
,'http://www.avante.pt/'
,'http://www.oje.pt/'
,'http://www.auniao.pt/'
,'http://www.record.pt/'
,'http://www.ojogo.pt/'
,'http://zerozero.pt/'
,'http://www.maisfutebol.iol.pt/'
,'http://desporto.sapo.pt/'
,'http://jornaleconomico.sapo.pt/'
,'http://www.diarioleiria.pt/'
,'http://www.regiaodeleiria.pt/'
,'http://www.correiodominho.pt/'
,'http://www.diariodominho.pt/'
,'http://economico.sapo.pt/']

Definimos o limite temporal para consulta do contamehistorias

In [4]:
params = { 'domains':domains, 
            'from':datetime(year=2010, month=1, day=1), 
            'to':datetime(year=2018, month=12, day=12) }

Exemplo dos dados que usamos do contamehistorias com a query "cristiano ronaldo"

In [ ]:
query="cristiano ronaldo" #query á escolha

apt =  ArquivoPT()
search_result = apt.getResult(query=query, **params)

language = "pt"

cont = engine.TemporalSummarizationEngine() #engine comtamehistorias
intervals = cont.build_intervals(search_result, language) #criação dos intervalos temporais com headlines
summ_result = cont.serialize(intervals) #serialize para ficar apenas com os dados bem formatados e com a informação mais relevante

for period in summ_result["results"]:
   
   print("--------------------------------")
   print(period["from"],"until",period["to"])
   result_interval =   {   'from':str(period['from']), 
                                        'to':str(period['to'])
                                    }
   # selected keyphrases
   keyphrases = period["keyphrases"]
   result_interval["headline"] = []
   for keyphrase in keyphrases:
       print(keyphrase["kw"])

Corremos o  Named Entity Recognition nos ultimos dados

In [ ]:
nlp = spacy.load('pt_core_news_sm') #modelo portugues do spacy
for keyphrase in keyphrases:
    doc = nlp(keyphrase["kw"])
    for ent in doc.ents:
       print(ent.text, ent.start_char, ent.end_char, ent.label_)

Agora adicionamos uma condição para apenas mostrar nomes de pessoas

In [ ]:
nlp = spacy.load('pt_core_news_sm') #modelo portugues do spacy
for keyphrase in keyphrases:
    doc = nlp(keyphrase["kw"])
    for ent in doc.ents:
        if ent.label_ == "PER": #condição apenas para nomes de pessoas
            print(ent.text, ent.start_char, ent.end_char, ent.label_)

Neste momento já temos tudo o que precisamos, temos os headlines e os nomes das pessoas isolados com as devidas posições no headline.
Passamos agora para a pesquisa das pessoas na API de imagens do ArquivoPt

In [ ]:
nlp = spacy.load('pt_core_news_sm')
for keyphrase in keyphrases:
    doc = nlp(keyphrase["kw"])
    for ent in doc.ents:
        if ent.label_ == "PER": #condição apenas para nomes de pessoas
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
            datetime_object = datetime.strptime(str(keyphrase["date"]), '%Y-%m-%d %H:%M:%S') #formatar a data para o formato pretendido
            datetime_objectMoreOne = datetime_object + relativedelta(years=+1) # adicionar um ano à data para pesquisa na api de imagens
            api_url_base = 'https://preprod.arquivo.pt/imagesearch/?' #url da API ArquivoPt
            query1 = ent.text #nome para query
            maxItems = 1 #maximo de resultados
            prettyPrint = 'true' #formatação bonita de resultados
            frm = datetime_object.strftime("%Y") #data formatada
            to = datetime_objectMoreOne.strftime("%Y")#data formatada +1 ano
            size = 'md' #tamanho das imagens 
            api_url = '{0}q={1}&maxItems={2}&prettyPrint={3}&to={4}&from={5}&size={6}'.format(api_url_base,query1,maxItems,prettyPrint,to,frm,size)
            print()
            print(api_url)
            response = requests.get(api_url) #resposta api de imagens
            imgApiResponse = response.json() #resposta em json api de imagens
            print(imgApiResponse)

Dados que nos recolhemos e usamos da API (em JSON)

In [ ]:
nlp = spacy.load('pt_core_news_sm')
newObject = []
for keyphrase in keyphrases:
    result_headline = {
                        'headline':str(keyphrase["kw"]),
                        'date':str(keyphrase["date"])
                    }
    result_headline["persons"] = []
    doc = nlp(keyphrase["kw"])
    for ent in doc.ents:
        if ent.label_ == "PER": #condição apenas para nomes de pessoas
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
            datetime_object = datetime.strptime(str(keyphrase["date"]), '%Y-%m-%d %H:%M:%S') #formatar a data para o formato pretendido
            datetime_objectMoreOne = datetime_object + relativedelta(years=+1) # adicionar um ano à data para pesquisa na api de imagens
            api_url_base = 'https://preprod.arquivo.pt/imagesearch/?' #url da API ArquivoPt
            query1 = ent.text #nome para query
            maxItems = 1 #maximo de resultados
            prettyPrint = 'true' #formatação bonita de resultados
            frm = datetime_object.strftime("%Y%m%d%H%M%S") #data formatada
            to = datetime_objectMoreOne.strftime("%Y%m%d%H%M%S")#data formatada +1 ano
            size = 'md' #tamanho das imagens 
            type = 'png jpg jpeg'
            api_url = '{0}q={1}&maxItems={2}&prettyPrint={3}&to={4}&from={5}&size={6}&type={7}'.format(api_url_base,query1,maxItems,prettyPrint,to,frm,size,type)
            response = requests.get(api_url) #resposta api de imagens
            imgApiResponse = response.json() #resposta em json api de imagens
            if(len(imgApiResponse["responseItems"]) > 0):
                result_chunk =   {   'person':str(ent.text), 
                                    'start_char':str(ent.start_char),
                                    'end_char':str(ent.end_char),
                                    'date':str(keyphrase["date"]),
                                    'imgSrc':str(imgApiResponse["responseItems"][0]["imgLinkToArchive"])
                                }
            else:
                result_chunk =   {   'person':str(ent.text), 
                                    'start_char':str(ent.start_char),
                                    'end_char':str(ent.end_char),
                                    'date':str(keyphrase["date"])
                                }
            result_headline["persons"].append(result_chunk)
    result_interval["headline"].append(result_headline)
    print(result_interval)

Dados que nos recolhemos e usamos da API (Forma Visual)

In [ ]:
nlp = spacy.load('pt_core_news_sm')
newObject = []
for keyphrase in keyphrases:
    result_headline = {
                        'headline':str(keyphrase["kw"]),
                        'date':str(keyphrase["date"])
                    }
    print("Headline: ",keyphrase["kw"])
    print("Data: ",keyphrase["date"])
    result_headline["persons"] = []
    doc = nlp(keyphrase["kw"])
    for ent in doc.ents:
        if ent.label_ == "PER": #condição apenas para nomes de pessoas
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
            datetime_object = datetime.strptime(str(keyphrase["date"]), '%Y-%m-%d %H:%M:%S') #formatar a data para o formato pretendido
            datetime_objectMoreOne = datetime_object + relativedelta(years=+1) # adicionar um ano à data para pesquisa na api de imagens
            api_url_base = 'https://preprod.arquivo.pt/imagesearch/?' #url da API ArquivoPt
            query1 = ent.text #nome para query
            maxItems = 1 #maximo de resultados
            prettyPrint = 'true' #formatação bonita de resultados
            frm = datetime_object.strftime("%Y%m%d%H%M%S") #data formatada
            to = datetime_objectMoreOne.strftime("%Y%m%d%H%M%S")#data formatada +1 ano
            size = 'md' #tamanho das imagens 
            type = 'png jpg jpeg'
            api_url = '{0}q={1}&maxItems={2}&prettyPrint={3}&to={4}&from={5}&size={6}&type={7}'.format(api_url_base,query1,maxItems,prettyPrint,to,frm,size,type)
            response = requests.get(api_url) #resposta api de imagens
            imgApiResponse = response.json() #resposta em json api de imagens
            if(len(imgApiResponse["responseItems"]) > 0):
                result_chunk =   {   'person':str(ent.text), 
                                    'start_char':str(ent.start_char),
                                    'end_char':str(ent.end_char),
                                    'date':str(keyphrase["date"]),
                                    'imgSrc':str(imgApiResponse["responseItems"][0]["imgLinkToArchive"])
                                }
                print("Nome: ",ent.text)
                print("Start Char: ",ent.start_char)
                print("End Char: ",ent.end_char)
                print("Data: ",keyphrase["date"])
                print("Link imagem: ",imgApiResponse["responseItems"][0]["imgLinkToArchive"])
            else:
                result_chunk =   {   'person':str(ent.text), 
                                    'start_char':str(ent.start_char),
                                    'end_char':str(ent.end_char),
                                    'date':str(keyphrase["date"])
                                }
                print("Nome: ",ent.text)
                print("Start Char: ",ent.start_char)
                print("End Char: ",ent.end_char)
                print("Data: ",keyphrase["date"])
            result_headline["persons"].append(result_chunk)
            print()
    result_interval["headline"].append(result_headline)

Em baixo esta tudo o que foi apresentado mas em forma de uma API propria para ser apresentado numa pagina de forma visual e com open cv a verificar se existem pessoas na foto

In [5]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	try:
		with urllib.request.urlopen(url) as response:
			image = np.asarray(bytearray(response.read()), dtype="uint8")
			image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	 
		# return the image
		return image
	except urllib.error.HTTPError as err:
		if err.code == 404:
			return None
		else:
			return image

class Search(Resource):
	def get(self, query):
		#query = 'Cristiano Ronaldo'
		fileName = 'results_opencv/{0}.txt'.format(query)
		fileNameFormated = 'results_opencv/{0}-formated.txt'.format(query)
		abs_fileName = 'results_opencv/{0}.txt'.format(query)
		abs_fileNameFormated = 'results_opencv/{0}-formated.txt'.format(query)
		faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
		#print(fileName)
		if not(os.path.isfile(abs_fileName)):
			apt =  ArquivoPT()
			search_result = apt.getResult(query=query, **params)

			language = "pt"
			
			cont = engine.TemporalSummarizationEngine()
			intervals = cont.build_intervals(search_result, language)

			f = open(abs_fileName, "w")
			f.write(json.dumps(cont.serialize(intervals)))
			#cont.pprint(intervals)
			f.close()
		else:
			print("Já existe")

		if not(os.path.isfile(abs_fileNameFormated)):
			f = open(abs_fileName, "r") 
			summ_result = json.loads(str(f.read()))
			nlp = spacy.load('pt_core_news_sm')
			newObject = []

			for period in summ_result["results"]:

				#print("--------------------------------")
				#print(period["from"],"until",period["to"])
				result_interval =   {   'from':str(period['from']), 
										'to':str(period['to'])
									}
				# selected keyphrases
				keyphrases = period["keyphrases"]
				result_interval["headline"] = []
				for keyphrase in keyphrases:  
					#print(keyphrase["kw"])
					result_headline = {
						'headline':str(keyphrase["kw"]),
						'date':str(keyphrase["date"])
					}
					result_headline["persons"] = []
					doc = nlp(keyphrase["kw"])
					for ent in doc.ents:
							if ent.label_ == "PER":
								#print(ent.text, ent.start_char, ent.end_char, ent.label_)
								datetime_object = datetime.strptime(str(keyphrase["date"]), '%Y-%m-%d %H:%M:%S')
								datetime_objectMoreOne = datetime_object + relativedelta(years=+1)
								api_url_base = 'https://arquivo.pt/imagesearch/?'
								query1 = ent.text
								maxItems = 10
								prettyPrint = 'true'
								frm = datetime_object.strftime("%Y%m%d%H%M%S")
								to = datetime_objectMoreOne.strftime("%Y%m%d%H%M%S")
								size = 'md'
								type = 'png jpg jpeg'
								api_url = '{0}q={1}&maxItems={2}&prettyPrint={3}&to={4}&from={5}&size={6}&type={7}'.format(api_url_base,query1,maxItems,prettyPrint,to,frm,size,type)
								#print(api_url)
								response = requests.get(api_url)
								imgApiResponse = response.json()
								#print(imgApiResponse)
								if(len(imgApiResponse["responseItems"]) > 0):
									for x in range(0, len(imgApiResponse["responseItems"])):
										#print(imgApiResponse["responseItems"][x]["imgLinkToArchive"])
										image = url_to_image(imgApiResponse["responseItems"][x]["imgLinkToArchive"])
										if image is not None:
											gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
											faces = faceCascade.detectMultiScale(
												gray,
												scaleFactor=1.2,
												minNeighbors=5,
												minSize=(30, 30),
												flags = cv2.CASCADE_SCALE_IMAGE
											)
											#print(len(faces))
											if(len(faces) >= 1): 
												#print("encontrou uma cara")
												result_chunk =   {   'person':str(ent.text), 
																	'start_char':str(ent.start_char),
																	'end_char':str(ent.end_char),
																	'date':str(keyphrase["date"]),
																	'imgSrc':str(imgApiResponse["responseItems"][x]["imgLinkToArchive"]),
																	'faces':str(len(faces))
																}
												break
											else:
												result_chunk =   {   'person':str(ent.text), 
																	'start_char':str(ent.start_char),
																	'end_char':str(ent.end_char),
																	'date':str(keyphrase["date"]),
																	'imgSrc':str('https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg')
																}
								else:
									result_chunk =   {   'person':str(ent.text), 
														'start_char':str(ent.start_char),
														'end_char':str(ent.end_char),
														'date':str(keyphrase["date"]),
														'imgSrc':str('https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg')
													}
								result_headline["persons"].append(result_chunk)
					result_interval["headline"].append(result_headline)
				newObject.append(result_interval)
			#print(json.dumps(newObject[0]))
			f.close()

			f = open(fileNameFormated, "w")
			f.write(json.dumps(newObject))
			return fileNameFormated, 200
				# sources
				# for headline in keyphrase["headlines"]:
				#    print("Date", headline["info"]["datetime"])
				#    print("Source", headline["info"]["domain"])
				#    print("Url", headline["info"]["url"])
					
				#  print()  
		else:
			return fileNameFormated, 200

@app.route("/")
def root():
	return render_template("index.html")

@app.route('/results_opencv/<path:path>')
def send_js(path):
	return send_from_directory('results_opencv', path)

In [ ]:
api.add_resource(Search, "/query/<string:query>")

app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2019 01:03:17] "GET / HTTP/1.1" 200 -


Já existe


127.0.0.1 - - [07/Apr/2019 01:03:19] "GET /query/cristiano%20ronaldo HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 01:03:19] "GET /results_opencv/cristiano%20ronaldo-formated.txt HTTP/1.1" 200 -
